In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import tensorflow as tf
import copy
import talib
import time
import requests
from statistics import mode
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

2023-09-24 20:35:02.378396: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
file_path = 'data (1).pkl'
df = pd.read_pickle(file_path)
stocks = pd.read_csv('stock_all.csv')
stocks = stocks.dropna(subset=['Sector']).reset_index()
mapping_dict = {'Industrials': 'manufacturing',
                'Real Estate': 'real_estate',
                'Miscellaneous': 'NaN',
                'Consumer Discretionary': 'retail_wholesale',
                'Finance': 'finance',
                'Health Care': 'life_sciences',
                'Consumer Staples': 'retail_wholesale',
                'Utilities': 'energy_transportation',
                'Technology': 'technology',
                'Energy': 'energy_transportation',
                'Telecommunications': 'technology',
                'Basic Materials': 'manufacturing'}

stocks['Topics'] = stocks['Sector'].map(mapping_dict)
stocks = stocks.drop(stocks[stocks['Topics'] == 'NaN'].index).reset_index()
stocks
data_all = df.merge(stocks, left_on='ticker', right_on='Symbol')
data_all

,Open,High,Low,Close,Adj Close,Volume_x,ticker,industry,level_0,index,...,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume_y,Sector,Industry,Topics
0,69.279999,69.410004,68.559998,68.849998,66.372200,1461100,A,life_sciences,0,0,...,$113.99,-2.27,-1.953%,3.335202e+10,United States,1999.0,1789767,Industrials,Biotechnology: Laboratory Analytical Instruments,manufacturing
1,69.639999,69.660004,69.000000,69.080002,66.593941,1816600,A,life_sciences,0,0,...,$113.99,-2.27,-1.953%,3.335202e+10,United States,1999.0,1789767,Industrials,Biotechnology: Laboratory Analytical Instruments,manufacturing
2,69.099998,69.269997,68.410004,68.930000,66.449326,1780400,A,life_sciences,0,0,...,$113.99,-2.27,-1.953%,3.335202e+10,United States,1999.0,1789767,Industrials,Biotechnology: Laboratory Analytical Instruments,manufacturing
3,69.040001,69.639999,68.879997,69.559998,67.056648,1183400,A,life_sciences,0,0,...,$113.99,-2.27,-1.953%,3.335202e+10,United States,1999.0,1789767,Industrials,Biotechnology: Laboratory Analytical Instruments,manufacturing
4,69.730003,71.160004,69.699997,71.129997,68.570168,2865600,A,life_sciences,0,0,...,$113.99,-2.27,-1.953%,3.335202e+10,United States,1999.0,1789767,Industrials,Biotechnology: Laboratory Analytical Instruments,manufacturing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5056415,6.960000,7.170000,6.880000,7.120000,7.120000,454900,ZYXI,life_sciences,5189,5616,...,$7.39,0.27,3.792%,2.654896e+08,United States,NaN,622488,Health Care,Biotechnology: Electromedical & Electrotherape...,life_sciences
5056416,7.110000,7.500000,6.920000,7.390000,7.390000,622500,ZYXI,life_sciences,5189,5616,...,$7.39,0.27,3.792%,2.654896e+08,United States,NaN,622488,Health Care,Biotechnology: Electromedical & Electrotherape...,life_sciences
5056417,7.710000,8.180000,7.620000,8.000000,8.000000,771900,ZYXI,life_sciences,5189,5616,...,$7.39,0.27,3.792%,2.654896e+08,United States,NaN,622488,Health Care,Biotechnology: Electromedical & Electrotherape...,life_sciences
5056418,8.000000,8.480000,7.980000,8.100000,8.100000,835700,ZYXI,life_sciences,5189,5616,...,$7.39,0.27,3.792%,2.654896e+08,United States,NaN,622488,Health Care,Biotechnology: Electromedical & Electrotherape...,life_sciences


In [ ]:
#count number of tickers under each industry

nan_count = data_all.isna().sum().sum()
print("Number of NaN Values in DataFrame:", nan_count)
unique_ticker_count = data_all['ticker'].nunique()
print("Number of Unique Tickers:", unique_ticker_count)
industry_ticker_counts = data_all.groupby('Topics')['ticker'].nunique().reset_index()
industry_ticker_counts = industry_ticker_counts.rename(columns={'ticker': 'unique_tickers_count'})
industry_ticker_counts

In [4]:
# try on a subset of the full data to reduce runtime

data_all = data_all[data_all['Topics'] == 'real_estate']
data_all

,Open,High,Low,Close,Adj Close,Volume_x,ticker,industry,level_0,index,...,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume_y,Sector,Industry,Topics
2514,10.05,10.120,10.03,10.030,7.841485,320900,AAIC,real_estate,2,2,...,$4.72,0.16,3.509%,138199547.0,United States,NaN,98323,Real Estate,Real Estate Investment Trusts,real_estate
2515,10.06,10.150,10.00,10.150,7.935300,288800,AAIC,real_estate,2,2,...,$4.72,0.16,3.509%,138199547.0,United States,NaN,98323,Real Estate,Real Estate Investment Trusts,real_estate
2516,10.13,10.150,10.04,10.140,7.927483,379400,AAIC,real_estate,2,2,...,$4.72,0.16,3.509%,138199547.0,United States,NaN,98323,Real Estate,Real Estate Investment Trusts,real_estate
2517,10.15,10.210,10.06,10.060,7.864938,380500,AAIC,real_estate,2,2,...,$4.72,0.16,3.509%,138199547.0,United States,NaN,98323,Real Estate,Real Estate Investment Trusts,real_estate
2518,10.10,10.140,10.05,10.120,7.911847,333500,AAIC,real_estate,2,2,...,$4.72,0.16,3.509%,138199547.0,United States,NaN,98323,Real Estate,Real Estate Investment Trusts,real_estate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4965771,2.05,2.240,1.61,1.650,1.650000,288700,WHLRP,real_estate,5086,5511,...,$1.74,0.09,5.455%,1706801.0,United States,NaN,6984,Real Estate,Real Estate Investment Trusts,real_estate
4965772,1.72,1.810,1.72,1.740,1.740000,7000,WHLRP,real_estate,5086,5511,...,$1.74,0.09,5.455%,1706801.0,United States,NaN,6984,Real Estate,Real Estate Investment Trusts,real_estate
4965773,1.62,1.765,1.55,1.765,1.765000,2400,WHLRP,real_estate,5086,5511,...,$1.74,0.09,5.455%,1706801.0,United States,NaN,6984,Real Estate,Real Estate Investment Trusts,real_estate
4965774,1.70,1.700,1.64,1.640,1.640000,1200,WHLRP,real_estate,5086,5511,...,$1.74,0.09,5.455%,1706801.0,United States,NaN,6984,Real Estate,Real Estate Investment Trusts,real_estate


In [6]:
class Stock:
    def __init__(self, data_all, ticker):
        self.data_all = data_all[data_all['ticker'] == ticker]
        self.ticker = ticker
        self.technical_indicators = pd.DataFrame()
        self.today_technical_indicators = {}
        self.labels = pd.DataFrame()
        self.prediction = 0.0

    def add_technical_indicators(self):
        prices = self.data_all.copy()
        prices["sma_10"] = talib.SMA(prices["Close"], timeperiod=10)
        prices["sma_50"] = talib.SMA(prices["Close"], timeperiod=50)
        prices["sma_200"] = talib.SMA(prices["Close"], timeperiod=200)
        prices["upper_band"], prices["middle_band"], prices["lower_band"] = talib.BBANDS(prices['Adj Close'])
        prices["RSI"] = talib.RSI(prices["Close"], timeperiod=14)
        prices["macd"], prices["macd_signal"], prices["macd_hist"] = talib.MACD(prices["Adj Close"])
        prices["stochastic_k"], prices["stochastic_d"] = talib.STOCH(prices["High"], prices["Low"], prices["Adj Close"])
        
        
        # Print the values of technical indicators for troubleshooting
        print("sma_10:", prices["sma_10"].iloc[-1])
        print("sma_50:", prices["sma_50"].iloc[-1])
        print("sma_200:", prices["sma_200"].iloc[-1])
        print("upper_band:", prices["upper_band"].iloc[-1])
        print("middle_band:", prices["middle_band"].iloc[-1])
        print("lower_band:", prices["lower_band"].iloc[-1])
        print("RSI:", prices["RSI"].iloc[-1])
        print("macd:", prices["macd"].iloc[-1])
        print("macd_signal:", prices["macd_signal"].iloc[-1])
        print("macd_hist:", prices["macd_hist"].iloc[-1])
        print("stochastic_k:", prices["stochastic_k"].iloc[-1])
        print("stochastic_d:", prices["stochastic_d"].iloc[-1])

        train_data_aux = prices[['Close', "sma_10", "sma_50", "sma_200", "upper_band", "middle_band", "lower_band",
                                "RSI", "macd", "macd_signal", "macd_hist", "stochastic_k", "stochastic_d"]].dropna()
        self.technical_indicators = train_data_aux.iloc[:-10, 1:]

        labels_aux = (train_data_aux['Close'].shift(-10) > train_data_aux['Close']).astype(int)
        self.labels = labels_aux[:-10]

        self.today_technical_indicators = prices[["sma_10", "sma_50", "sma_200", "upper_band", "middle_band",
                                                   "lower_band", "RSI", "macd", "macd_signal", "macd_hist",
                                                   "stochastic_k", "stochastic_d"]].iloc[-1]



def filter_technical_indicator(stock, indicator_name, operator, value):
    if indicator_name not in stock.today_technical_indicators:
        print(f"{indicator_name} not found for {stock.ticker}")
        return False

        # Obtain the value of the technical indicator
    indicator_value = stock.today_technical_indicators[indicator_name]
    
        
    if value == 'Close':
        value = float(stock.data_all['Close'].iloc[-1])  # Use the 'Close' price as the current price
    else:
        value = float(value)
    
        # Compare according to operator
    if operator == '>':
        return float(indicator_value) > value
    elif operator == '>=':
        return float(indicator_value) >= value
    elif operator == '<':
        return float(indicator_value) < value
    elif operator == '<=':
        return float(indicator_value) <= value
    elif operator == '==':
        return float(indicator_value) == value
    else:
        return False


class StockScreener:
    def __init__(self, stocks, filters):
        self.stocks = stocks
        self.filters = filters
        self.models = {}

    def apply_filters(self):
        filtered_stocks = []
        for stock in self.stocks:
            passed_all_filters = True
            for filter_func in self.filters:
                if not filter_func(stock):
                    passed_all_filters = False
                    print(f"Filter {filter_func.__name__} failed for {stock.ticker}")
                    break
            if passed_all_filters:
                filtered_stocks.append(stock)
        return filtered_stocks

    def create_model(self, train_data):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(64, input_shape=(train_data.shape[1],), activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def train_models(self):
        for stock in self.stocks:
            stock.add_technical_indicators()
            train_data = stock.technical_indicators
            train_labels = stock.labels

            scaler = StandardScaler()
            train_data = scaler.fit_transform(train_data)
            train_labels = np.array(train_labels)

            model = self.create_model(train_data)
            model.fit(train_data, train_labels, epochs=10)
            self.models[stock.ticker] = (model, scaler)

    def predict_stocks(self, new_stocks):
        predicted_stocks = []
        for stock in new_stocks:
            if stock.ticker in self.models:
                model, scaler = self.models[stock.ticker]
                new_features_aux = np.array(stock.today_technical_indicators).reshape(1, -1)
                new_stock_data = scaler.transform(new_features_aux)
                prediction = model.predict(new_stock_data)
                stock.prediction = prediction
                if prediction > 0.5:
                    predicted_stocks.append(stock)

        return predicted_stocks




In [12]:
# Define the required filters here for each technical indicator

filters = [
    #lambda stock: filter_technical_indicator(stock, 'UpperBand', '>', 'price'),
    #lambda stock: filter_technical_indicator(stock, 'LowerBand', '<', 'price'),
    lambda stock: stock.today_technical_indicators.get('RSI', 0) > 50
]

# Create instances of the Stock class with data_all and tickers
all_stocks = [Stock(data_all, ticker) for ticker in data_all['ticker'].unique()]

# Initialize the screener with stocks and filters
screener = StockScreener(all_stocks, filters)

# Apply Filters
filtered_stocks = screener.apply_filters()

# Print the values of technical indicators for each stock
for stock in filtered_stocks:
    stock.add_technical_indicators()

# Print the tickers of filtered stocks
filtered_stock_tickers = [stock.ticker for stock in filtered_stocks]
print("Filtered Stocks Tickers:", filtered_stock_tickers)

# Train model
screener.train_models()

# Make predictions
predicted_stocks = screener.predict_stocks(filtered_stocks)

# Print the tickers of predicted stocks
#predicted_stock_tickers = [stock.ticker for stock in predicted_stocks]
#print("Predicted Stocks Tickers:", predicted_stock_tickers)

#filtered_stocks, predicted_stocks


Filter <lambda> failed for AAIC
Filter <lambda> failed for AAIN
Filter <lambda> failed for AAT
Filter <lambda> failed for ABR
Filter <lambda> failed for ACR
Filter <lambda> failed for ACRE
Filter <lambda> failed for ADC
Filter <lambda> failed for AHT
Filter <lambda> failed for AIC
Filter <lambda> failed for AIRC
Filter <lambda> failed for AIV
Filter <lambda> failed for AJX
Filter <lambda> failed for AJXA
Filter <lambda> failed for AKR
Filter <lambda> failed for ALEX
Filter <lambda> failed for ALX
Filter <lambda> failed for AMH
Filter <lambda> failed for AMT
Filter <lambda> failed for AOMR
Filter <lambda> failed for APLE
Filter <lambda> failed for ARE
Filter <lambda> failed for ARI
Filter <lambda> failed for ARR
Filter <lambda> failed for AVB
Filter <lambda> failed for AXR
Filter <lambda> failed for BDN
Filter <lambda> failed for BFS
Filter <lambda> failed for BHR
Filter <lambda> failed for BNL
Filter <lambda> failed for BRSP
Filter <lambda> failed for BRT
Filter <lambda> failed for BRX

In [13]:
# Calculate and store RSI values for all stocks
rsi_data = []

for stock in all_stocks:
    stock.add_technical_indicators()
    rsi_value = stock.today_technical_indicators.get('RSI', 0)
    rsi_data.append({'Ticker': stock.ticker, 'RSI': rsi_value})

rsi_df = pd.DataFrame(rsi_data)

# Find the maximum and minimum RSI values in the DataFrame
max_rsi = rsi_df['RSI'].max()
min_rsi = rsi_df['RSI'].min()

print(f"Maximum RSI Value: {max_rsi}")
print(f"Minimum RSI Value: {min_rsi}")
rsi_df

sma_10: 4.6
sma_50: 4.568200044631958
sma_200: 3.4599999976158142
upper_band: 4.730245411528015
middle_band: 4.607999992370606
lower_band: 4.485754573213196
RSI: 50.31093398578474
macd: 0.0291664575526438
macd_signal: 0.02508088251133893
macd_hist: 0.00408557504130487
stochastic_k: 28.985546306757687
stochastic_d: 44.806732195274485
sma_10: 23.555299949645995
sma_50: 23.649239807128907
sma_200: 23.3367050075531
upper_band: 23.634907629903427
middle_band: 23.47460021972656
lower_band: 23.314292809549695
RSI: 44.14739473256141
macd: -0.03321168875038438
macd_signal: -0.005886029743805987
macd_hist: -0.027325659006578396
stochastic_k: 52.4483199562263
stochastic_d: 60.9599989694066
sma_10: 20.854999923706053
sma_50: 21.340800094604493
sma_200: 22.193699998855593
upper_band: 20.780136993751462
middle_band: 20.487999725341798
lower_band: 20.195862456932133
RSI: 36.14992352496997
macd: -0.15668410604175875
macd_signal: -0.09830781365485883
macd_hist: -0.058376292386899925
stochastic_k: 23.29

,Ticker,RSI
0,AAIC,50.310934
1,AAIN,44.147395
2,AAT,36.149924
3,ABR,46.395691
4,ACR,38.395647
...,...,...
225,UNIT,40.101662
226,VINO,25.134913
227,WHLR,38.990901
228,WHLRD,38.822474


In [14]:
# Apply Filters
filtered_stocks = screener.apply_filters()
# Print the tickers of stocks that passed the filter
passed_stock_tickers = [stock.ticker for stock in filtered_stocks]
print("Stocks that passed the filter:", passed_stock_tickers)
num_passed = sum(1 for stock in filtered_stocks )
print("Number of stocks that passed the filter:", num_passed)



Filter <lambda> failed for AAIN
Filter <lambda> failed for AAT
Filter <lambda> failed for ABR
Filter <lambda> failed for ACR
Filter <lambda> failed for ADC
Filter <lambda> failed for AHT
Filter <lambda> failed for AIC
Filter <lambda> failed for AIRC
Filter <lambda> failed for AIV
Filter <lambda> failed for AJXA
Filter <lambda> failed for ALEX
Filter <lambda> failed for AMT
Filter <lambda> failed for AOMR
Filter <lambda> failed for ARE
Filter <lambda> failed for ARI
Filter <lambda> failed for ARR
Filter <lambda> failed for AVB
Filter <lambda> failed for BDN
Filter <lambda> failed for BFS
Filter <lambda> failed for BHR
Filter <lambda> failed for BNL
Filter <lambda> failed for BRSP
Filter <lambda> failed for BRT
Filter <lambda> failed for BXP
Filter <lambda> failed for CBL
Filter <lambda> failed for CCI
Filter <lambda> failed for CHCT
Filter <lambda> failed for CLPR
Filter <lambda> failed for CMTG
Filter <lambda> failed for COLD
Filter <lambda> failed for CORR
Filter <lambda> failed for C